In [2]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels
using LaTeXTabulars

In [8]:
Random.seed!(02102023);

# Simulate data
Let $\eta_t \sim N(0,1)$ i.i.d.; $\epsilon_{it} \sim N(0,1)$ i.i.d.; $x_t \sim \exp(N(0,1))$ i.i.d.; $z_{it} \sim N(0,2)$ i.i.d.

Let $K=30$ and $T=100$

True parameters:

- $\alpha = 1$
- $\beta = 2$ 
- $\delta = 6$ 
- $\gamma = 3$ 
- $\rho = 0.8$

In [11]:
# parameters 
α = 1; 
β = 2;
δ = 6;
γ = 3; 
ρ = 0.8;

# markets and firms
K = 30; 
T = 100; 

# specify distributions 
normal_0_1 = Normal(0,1);
lognormal_0_1 = LogNormal(0,1);
normal_0_2 = Normal(0, 2);

In [36]:
# market level 
market_df = DataFrame(market = 1:T,
    ηₜ = rand(normal_0_1, T), 
    xₜ = rand(lognormal_0_1, T));

# firm-market level
firm_market_df = DataFrame(firm = repeat(1:K, inner=T),
    market = repeat(1:T, outer=K),
    ϵᵢₜ = rand(normal_0_1, K * T),
    zᵢₜ = rand(normal_0_2, K * T));
firm_market_df = innerjoin(firm_market_df, market_df, on=:market)

# sort by heterogenous profitability in a market
sort!(firm_market_df, [:market, :firm])

,firm,market,ϵᵢₜ,zᵢₜ,ηₜ,xₜ
,Int64,Int64,Float64,Float64,Float64,Float64
1,1,1,0.0160862,1.80244,1.08928,1.24193
2,1,2,-0.22144,-1.78475,-0.877599,0.533368
3,1,3,-1.8665,-4.0658,-1.05373,0.39351
4,1,4,0.127258,-1.21048,-1.53024,1.35369
5,1,5,-0.574106,-3.34153,-1.02966,7.43488
6,1,6,2.15781,-3.60995,-2.45399,1.32156
7,1,7,-0.509222,-2.64191,-0.288123,1.1871
8,1,8,1.10211,2.41379,1.10304,1.95764
9,1,9,1.22902,0.508983,1.00609,0.919765


In [37]:
market_df

,market,ηₜ,xₜ
,Int64,Float64,Float64
1,1,1.08928,1.24193
2,2,-0.877599,0.533368
3,3,-1.05373,0.39351
4,4,-1.53024,1.35369
5,5,-1.02966,7.43488
6,6,-2.45399,1.32156
7,7,-0.288123,1.1871
8,8,1.10304,1.95764
9,9,1.00609,0.919765
